In [ ]:
pip install PyPDF2 nltk numpy

In [ ]:
import PyPDF2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np

In [ ]:
#doing preprocessing and creating word_embedding of words
import PyPDF2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np

# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

# Step 2: Preprocess the text
def preprocess_text(text):
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('wordnet')
    
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    words = word_tokenize(text.lower())
    words = [word for word in words if word.isalpha() and word not in stop_words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    return lemmatized_words

# Step 3: Load pre-trained GloVe embeddings
def load_glove_embeddings(glove_path):
    embeddings_index = {}
    with open(glove_path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
            except ValueError:
                print(f"Skipping line with word '{word}' due to conversion error.")
    return embeddings_index

# Step 4: Create word embeddings dictionary
def create_word_embeddings_dictionary(words, embeddings_index):
    word_embeddings = {word: embeddings_index[word] for word in words if word in embeddings_index}
    return word_embeddings

# Path to your PDF file

pdf_path=r"C:\Users\chand\Desktop\IIT MADRAS INTERNSHIP\Transcripts\ALL PDF TOGETHER\merged.pdf"

# Path to GloVe embeddings file

glove_path=r"C:\Users\chand\Desktop\IIT MADRAS INTERNSHIP\Transcripts\glove.840B.300d.txt"

# Extract, preprocess and generate embeddings
text = extract_text_from_pdf(pdf_path)
lemmatized_words = preprocess_text(text)
glove_embeddings = load_glove_embeddings(glove_path)
word_embeddings = create_word_embeddings_dictionary(lemmatized_words, glove_embeddings)

# Display the word embeddings dictionary
print(word_embeddings)


In [ ]:
pip install scikit-learn

In [ ]:
pip install matplotlib

In [ ]:
##performing pca
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
def perform_pca(word_embeddings):
    words = list(word_embeddings.keys())
    vectors = np.array(list(word_embeddings.values()))

    pca = PCA(n_components=2)
    reduced_vectors = pca.fit_transform(vectors)
    
    return words, reduced_vectors
# Step 6: Plot words in 2D space
def plot_words_in_2d(words, reduced_vectors):
    plt.figure(figsize=(10, 10))
    plt.scatter(reduced_vectors[:, 0], reduced_vectors[:, 1], color='red')

    for i, word in enumerate(words):
        plt.annotate(word, (reduced_vectors[i, 0], reduced_vectors[i, 1]))
    
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('Word Embeddings PCA')
    plt.show()

In [ ]:
# Perform PCA and plot the words
words, reduced_vectors = perform_pca(word_embeddings)
plot_words_in_2d(words, reduced_vectors)

In [ ]:
# Take first 100 words and their corresponding reduced vectors
words_100 = words[:100]
reduced_vectors_100 = reduced_vectors[:100]

# Plot the first 100 words in 2D space
plt.figure(figsize=(10, 10))
plt.scatter(reduced_vectors_100[:, 0], reduced_vectors_100[:, 1], color='red')

for i, word in enumerate(words_100):
    plt.annotate(word, (reduced_vectors_100[i, 0], reduced_vectors_100[i, 1]))

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Word Embeddings PCA (First 100 Words)')
plt.show()


In [ ]:
pip install pandas
pip install openpyxl

In [ ]:
import pandas as pd

question=pd.read_excel(r"C:\Users\chand\Desktop\IIT MADRAS INTERNSHIP\Transcripts\students_questions.xlsx")

In [ ]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Sample data (trainning data which were manually labelle for training purpose)
data = {
    "Question": [
        "What is the space complexity of a DataFrame?",
        "What is the time complexity of sorting a DataFrame?",
        "What is the time complexity of sorting a DataFrame?",
        "What is the time complexity of groupby function in a DataFrame?",
        "What is the time complexity of concatenate function in a DataFrame?",
        "Are there any vectorized operations in Pandas DataFrame columns?",
        "Pandas",
        "Comparing two one-dimensional arrays",
        "How to handle multiple files concurrently?",
        "Fastest way to compare two arrays?",
        "Using threads for handling multiple files...using Python?",
        "Comparing two numpy arrays",
        "How much time does it take to read a JSON file?",
        "Efficient methods for creating directories and processing JSON files?",
        "Dictionary storage vs list storage: which is faster in access time?",
        "Time complexity in adding new column in a DataFrame?",
        "How can I improve performance when using groupby in Pandas DataFrame?",
        "What are the recommended practices for handling exceptions in Python?",
        "Optimizing File Handling and Directory Creation in Python",
        "How is searching in dictionaries done?",
        "Time complexity of searching in a dictionary",
        "How can Python handle processing multiple files concurrently?",
        "What are the advantages of using threads for concurrent file processing?",
        "What are some best practices for optimizing performance in Python?"
    ],
    "Solo Class": [
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Relational",
        "Unistructural",
        "Unistructural",
        "Unistructural",
        "Unistructural",
        "Multistructural",
        "Unistructural",
        "Unistructural",
        "Multistructural",
        "Relational",
        "Multistructural",
        "Relational",
        "Extended Abstract",
        "Relational",
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Extended Abstract"
    ]
}

question_df = pd.DataFrame(data)


# Function to get the average embedding for a question
def get_question_embedding(question, word_embeddings):
    words = word_tokenize(question.lower())
    valid_embeddings = [word_embeddings[word] for word in words if word in word_embeddings]
    if valid_embeddings:
        return np.mean(valid_embeddings, axis=0)
    else:
        return np.zeros(next(iter(word_embeddings.values())).shape)

# Create embeddings for each question
question_df['Embeddings'] = question_df['Question'].apply(lambda x: get_question_embedding(x, word_embeddings))
X = np.vstack(question_df['Embeddings'].values)


In [ ]:
pip install rake_nltk

In [ ]:
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords

# Ensure necessary NLTK data is downloaded
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Function to fetch questions from Stack Exchange API
def fetch_questions_from_stack_exchange(tags, pagesize=100, max_pages=20):
    questions = []
    base_url = 'https://api.stackexchange.com/2.3/questions'
    for page in range(1, max_pages + 1):
        params = {
            'order': 'desc',
            'sort': 'creation',
            'tagged': tags,
            'site': 'stackoverflow',
            'pagesize': pagesize,
            'page': page
        }
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Error: Unable to fetch data (status code {response.status_code})")
            continue
        
        data = response.json()
        for item in data['items']:
            questions.append(item['title'])
    
    return questions

# Tags related to PDSA
tags = 'python;data-structures;algorithms'

# Fetch questions for the specified tags
all_questions = fetch_questions_from_stack_exchange(tags, max_pages=20)

# Create a DataFrame with the fetched questions
questions_df = pd.DataFrame(all_questions, columns=['Question'])

# Labeling function
def label_question(question):
    if any(keyword in question.lower() for keyword in ['complexity', 'performance']):
        return 'Multistructural'
    elif any(keyword in question.lower() for keyword in ['compare', 'difference']):
        return 'Relational'
    elif any(keyword in question.lower() for keyword in ['how', 'what', 'why']):
        return 'Unistructural'
    elif any(keyword in question.lower() for keyword in ['best practices', 'optimization']):
        return 'Extended Abstract'
    else:
        return 'Prestructural'



rake = Rake()


def extract_keywords_with_rake(text):
    rake.extract_keywords_from_text(text)
    keywords = rake.get_ranked_phrases()[:5]  # Extract top 5 keywords
    # Join keywords into a single string separated by commas
    clean_keywords = ', '.join(keywords)
    return clean_keywords

# Extract keywords for each question and store them in the ImportantKeywords column
#questions_df['ImportantKeywords'] = extract_keywords_with_rake(questions_df['Question'])
questions_df['ImportantKeywords']=questions_df['Question'].apply(extract_keywords_with_rake)

# Print the DataFrame with ImportantKeywords
#print(questions_df.head())

# Assuming question_df is your existing DataFrame
combined_df = pd.concat([question_df, questions_df], ignore_index=True)

# Print the combined DataFrame
#print(combined_df.head())


In [ ]:
combined_df

In [ ]:
import pandas as pd

data = {
    "Question": [
        "What is the space complexity of a DataFrame?",
        "What is the time complexity of sorting a DataFrame?",
        "What is the time complexity of sorting a DataFrame?",
        "What is the time complexity of groupby function in a DataFrame?",
        "What is the time complexity of concatenate function in a DataFrame?",
        "Are there any vectorized operations in Pandas DataFrame columns?",
        "Pandas",
        "Comparing two one-dimensional arrays",
        "How to handle multiple files concurrently?",
        "Fastest way to compare two arrays?",
        "Using threads for handling multiple files...using Python?",
        "Comparing two numpy arrays",
        "How much time does it take to read a JSON file?",
        "Efficient methods for creating directories and processing JSON files?",
        "Dictionary storage vs list storage: which is faster in access time?",
        "Time complexity in adding new column in a DataFrame?",
        "How can I improve performance when using groupby in Pandas DataFrame?",
        "What are the recommended practices for handling exceptions in Python?",
        "Optimizing File Handling and Directory Creation in Python",
        "How is searching in dictionaries done?",
        "Time complexity of searching in a dictionary",
        "How can Python handle processing multiple files concurrently?",
        "What are the advantages of using threads for concurrent file processing?",
        "What are some best practices for optimizing performance in Python?",
        "Efficient Merge sort implementation in python",
        "Does a recursive segment tree require more space than an iterative one?",
        "Implementing a FIFO allocation between two lists in Python",
        "Leetcode - Combination sum problem in Python",
        "Maze solving algorithm gets stuck and keeps running in \"circles\"",
        "Branch and bound algorithm to find the shortest path with constraints",
        "Find words with only one letter difference from a list of words",
        "Algorithm for compound fractions",
        "Rotate an array in-place",
        "Why I am getting MLE(memory limit exceeded) in hackerearth platform",
        "Hire K workers test case fail",
        "Strategies for Enhancing Algorithm Efficiency?",
        "Minimum window substring not sliding correctly",
        "Calculate the all the possible difference between elements of a list in an optimized way",
        "How can I efficiently sort a large dataset in Python?",
        "Time limit exceeded on the code: Check If a String Contains All Binary Codes of Size K",
        "Implementing a max heap in python using heapq for a custom class object with custom comparators in Python?",
        "Edge case of binary subarray with sum",
        "Number of hits in Fibonacci using dynamic programming",
        "How does this seemingly random bit of string solve the TwoSum problem?",
        "Recreate a O(n*k) algorithm to Θ(n)",
        "Can I further speed up query to find all objects within a range in python",
        "An algorithm to find the shortest path based on 2 criteria",
        "Flatten Binary Tree explanation",
        "A probabilistic data structure based on flipping bits with probability `1/2^x` for counting",
        "Binary Tree Max Path Sum: Faulty Logic",
        "How to increment each sublist in a given list by a step of one?",
        "Splitting binary tree code does not work on some cases",
        "Why does interpolation search calculate the position same as linear search",
        "How to split a tree in branches based on depth",
        "Using clear() vs copy() in a hashset",
        "LeetCode 2467: Most Profitable Path in a Tree",
        "Generate list of pattern contained in dictionary values Python",
        "Depth First Search Logic Recursive Function",
        "Solution of maze",
        "Create a hierarchy data structure from list of data",
        "What is a more optimal algorithm for this problem?",
        "Using prefix sum to calculate scores of 2 robots in a 2d array",
        "Optimizing a geometrical algorithm [ points inside a region inside a circle ]",
        "How to use mergesort on a list of tuples in Python?",
        "Number of permutation operations to return to original array",
        "Make Pairs Of Users using python",
        "Does a solution better than O(N^2) exist Python",
        "Extract values from json data with condition and add them in as new columns",
        "Finding the minimum absolute difference between elements minimum of k distance apart",
        "Why am I getting error in this K-Nearest Neighbour algorithm?",
        "Efficiently Identifying non-overlapping Intervals in a list of types in. python",
        "How to perform mathematical operations on the value of an existing dictionary?",
        "How to add frequency of different letters for different positions across words in a list?",
        "Python Dictionary Search is taking long time",
        "How to generate a unique set of 3 digit numbers from an array",
        "How to retrieve specific matches from Trie search results in Python?",
        "I am trying to output data in my terminal upon a tkinter button click, I am getting this error but I am not sure where I am going wrong",
        "How could I improve my data structure for superpixel coordinates storage in image segmentation algorithms?",
        "Submission and custom input on GeeksForGeeks gives different judge result on same test case",
        "someone help me understand what is constant factor of a computer and what is mean by \"faster computer have low constant factor\""
    ],
    "Label": [
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Prestructural",
        "Relational",
        "Relational",
        "Multistructural",
        "Relational",
        "Multistructural",
        "Unistructural",
        "Multistructural",
        "Relational",
        "Multistructural",
        "Relational",
        "Relational",
        "Relational",
        "Relational",
        "Multistructural",
        "Relational",
        "Relational",
        "Relational",
        "Multistructural",
        "Relational",
        "Multistructural",
        "Relational",
        "Prestructural",
        "Relational",
        "Unistructural",
        "Unistructural",
        "Unistructural",
        "Prestructural",
        "Unistructural",
        "Relational",
        "Multistructural",
        "Relational",
        "Relational",
        "Unistructural",
        "Multistructural",
        "Multistructural",
        "Multistructural",
        "Unistructural",
        "Extended Abstract",
        "Multistructural",
        "Relational",
        "Unistructural",
        "Extended Abstract",
        "Prestructural",
        "Unistructural",
        "Unistructural",
        "Multistructural",
        "Unistructural",
        "Prestructural",
        "Unistructural",
        "Relational",
        "Unistructural",
        "Unistructural",
        "Relational",
        "Multistructural",
        "Unistructural",
        "Unistructural",
        "Multistructural",
        "Unistructural",
        "Multistructural",
        "Extended Abstract",
        "Unistructural",
        "Multistructural",
        "Unistructural",
        "Multistructural",
        "Unistructural",
        "Unistructural",
        "Prestructural",
        "Unistructural",
        "Unistructural",
        "Prestructural",
        "Extended Abstract",
        "Prestructural",
        "Unistructural"
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display DataFrame
print(df)



In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords

# Ensure the NLTK stopwords are downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))





# Preprocess text to remove stop words
def preprocess_text(text):
    words = text.split()
    return ' '.join([word for word in words if word.lower() not in stop_words])

# Preprocess the questions
df['PreprocessedQuestion'] = df['Question'].apply(preprocess_text)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['PreprocessedQuestion'])
feature_names = tfidf_vectorizer.get_feature_names_out()


rake = Rake()

# Extract keywords using RAKE and clean the output
def extract_keywords_with_rake(text):
    rake.extract_keywords_from_text(text)
    keywords = rake.get_ranked_phrases()[:5]  # Extract top 5 keywords
    # Split concatenated keywords
    clean_keywords = []
    for kw in keywords:
        clean_keywords.extend(kw.split())
    return clean_keywords


#df['ImportantKeywords'] = [extract_keywords(tfidf_matrix[row], feature_names) for row in range(tfidf_matrix.shape[0])]

df['ImportantKeywords'] = df['PreprocessedQuestion'].apply(extract_keywords_with_rake)


# Calculate average distance between keywords
def calculate_average_distance(keywords, embeddings):
    vectors = [embeddings[word] for word in keywords if word in embeddings]
    if len(vectors) < 2:
        return 0  # Not enough keywords to calculate distance
    
    distances = euclidean_distances(vectors)
    total_distance = 0
    count = 0
    
    for i in range(len(vectors)):
        for j in range(i + 1, len(vectors)):
            total_distance += distances[i, j]
            count += 1
    
    if count == 0:
        return 0  # To avoid division by zero if no pairs are found
    
    return total_distance / count

df['AvgDistance'] = df['ImportantKeywords'].apply(lambda x: calculate_average_distance(x, word_embeddings))

# Calculate average distance for each category
category_avg_distances = df.groupby('Label')['AvgDistance'].mean().to_dict()

#import ace_tools as tools; tools.display_dataframe_to_user(name="Questions with Average Distances", dataframe=df)

print("Category Average Distances:\n", category_avg_distances)
#print(df[['Question', 'Label', 'ImportantKeywords', 'AvgDistance']])


In [ ]:
all_questions=combined_df['Question'].tolist()

In [ ]:
only_question_df=questions_df['Question']

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import euclidean_distances
from rake_nltk import Rake

# Ensure the NLTK stopwords are downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))




# Preprocess text to remove stop words
def preprocess_text(text):
    words = text.split()
    return ' '.join([word for word in words if word.lower() not in stop_words])

# Preprocess the new questions
new_df = pd.DataFrame(all_questions, columns=['Question'])
new_df['PreprocessedQuestion'] = new_df['Question'].apply(preprocess_text)

# Initialize RAKE keyword extractor
rake = Rake()

# Extract keywords using RAKE and clean the output
def extract_keywords_with_rake(text):
    rake.extract_keywords_from_text(text)
    keywords = rake.get_ranked_phrases()[:5]  # Extract top 5 keywords
    # Split concatenated keywords
    clean_keywords = []
    for kw in keywords:
        clean_keywords.extend(kw.split())
    return clean_keywords

new_df['ImportantKeywords'] = new_df['PreprocessedQuestion'].apply(extract_keywords_with_rake)

# Calculate average distance between keywords
def calculate_average_distance(keywords, embeddings):
    vectors = [embeddings[word] for word in keywords if word in embeddings]
    if len(vectors) < 2:
        return 0  # Not enough keywords to calculate distance
    
    distances = euclidean_distances(vectors)
    total_distance = 0
    count = 0
    
    for i in range(len(vectors)):
        for j in range(i + 1, len(vectors)):
            total_distance += distances[i, j]
            count += 1
    
    if count == 0:
        return 0  # To avoid division by zero if no pairs are found
    
    return total_distance / count

new_df['AvgDistance'] = new_df['ImportantKeywords'].apply(lambda x: calculate_average_distance(x, word_embeddings))

# Thresholds from previously computed data
thresholds ={'Extended Abstract': 7.346035873889923, 'Multistructural': 7.234015334291118, 'Prestructural': 6.328144119996013, 'Relational': 7.291196525891623, 'Unistructural': 7.088332765060241}

# Function to classify new questions
def classify_question(avg_distance, thresholds):
    closest_category = min(thresholds.keys(), key=lambda k: abs(thresholds[k] - avg_distance))
    return closest_category

# Classify each new question
new_df['PredictedClass'] = new_df['AvgDistance'].apply(lambda x: classify_question(x, thresholds))

In [ ]:
##some EDA

In [ ]:
l1=new_df['PredictedClass'].tolist()
new_dict={}
for element in l1:
    if element not in new_dict:
        new_dict[element]=1
    else:
        new_dict[element]=new_dict[element]+1
new_dict  

In [ ]:
import matplotlib.pyplot as plt

# Data for the bar graph
categories = ['Multistructural', 'Extended Abstract', 'Prestructural','Relational','Unistructural']
counts = [34,487,136,27,142]

# Plotting the bar graph
plt.figure(figsize=(10, 6))
plt.bar(categories, counts, color=['blue', 'green', 'red','yellow','purple'])
plt.xlabel('Question Categories')
plt.ylabel('Number of Questions')
plt.title('Number of Questions by Category')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# Assuming 'df' is your DataFrame
# Create a list of all keywords
all_keywords = [keyword for sublist in new_df['ImportantKeywords'] for keyword in sublist]

# Count the frequency of each keyword
keyword_counts = Counter(all_keywords)

# Get the top 50 keywords
top_50_keywords = dict(keyword_counts.most_common(50))

# Plot the top 50 keywords
plt.figure(figsize=(12, 8))
plt.bar(top_50_keywords.keys(), top_50_keywords.values())
plt.xlabel('Keywords')
plt.ylabel('Frequency')
plt.title('Top 50 Keywords by Frequency')
plt.xticks(rotation=90)
plt.show()


In [ ]:
new_questions=pd.read_csv(r"C:\Users\chand\Desktop\IIT MADRAS INTERNSHIP\Transcripts\QueryResults.csv")

In [ ]:
new_questions= new_questions.drop(columns=['Post Link','User Link','CreationDate'])

In [ ]:
new_list=new_questions['Title'].tolist()

In [ ]:
new_questions['PreprocessedQuestion'] = new_questions['Title'].apply(preprocess_text)
new_questions['ImportantKeywords'] = new_questions['PreprocessedQuestion'].apply(extract_keywords_with_rake)
new_questions['AvgDistance'] = new_questions['ImportantKeywords'].apply(lambda x: calculate_average_distance(x, word_embeddings))
new_questions['PredictedClass'] = new_questions['AvgDistance'].apply(lambda x: classify_question(x, thresholds))
new_questions.head()

In [ ]:
l2=new_questions['PredictedClass'].tolist()
new_dict1={}
for element in l2:
    if element not in new_dict1:
        new_dict1[element]=1
    else:
        new_dict1[element]=new_dict1[element]+1
new_dict1 

In [ ]:
import matplotlib.pyplot as plt

# Data for the bar graph
categories =list(new_dict1.keys())
counts =list(new_dict1.values())

# Plotting the bar graph
plt.figure(figsize=(10, 6))
plt.bar(categories, counts, color=['blue', 'green', 'red','yellow','purple'])
plt.xlabel('Question Categories')
plt.ylabel('Number of Questions')
plt.title('Number of Questions by Category')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# Assuming 'df' is your DataFrame
# Create a list of all keywords
all_keywords = [keyword for sublist in new_questions['ImportantKeywords'] for keyword in sublist]

# Count the frequency of each keyword
keyword_counts = Counter(all_keywords)

# Get the top 50 keywords
top_50_keywords = dict(keyword_counts.most_common(50))

# Plot the top 50 keywords
plt.figure(figsize=(12, 8))
plt.bar(top_50_keywords.keys(), top_50_keywords.values())
plt.xlabel('Keywords')
plt.ylabel('Frequency')
plt.title('Top 50 Keywords by Frequency')
plt.xticks(rotation=90)
plt.show()

In [ ]:
l3=new_questions['AnswerCount'].tolist()
new_dict2={}
for element in l3:
    if element not in new_dict2:
        new_dict2[element]=1
    else:
        new_dict2[element]=new_dict2[element]+1
new_dict2

In [ ]:
categories = list(new_dict2.keys())
values = list(new_dict2.values())

# Creating the bar plot
plt.bar(categories, values)

# Adding labels and title
plt.xlabel('Number of Answers')
plt.ylabel('Nunber of Count')
plt.title('Number of count for each number of Answers')

# Display the plot
plt.show()

In [ ]:
l4=new_questions['ViewCount'].tolist()
l5=new_questions['PredictedClass'].tolist()
new_set=set(l5)
l6=list(new_set)
dict4=dict()
for element in l6:
    dict4[element]=0
n=len(l4)
for i in range(n):
    dict4[l5[i]]=dict4[l5[i]]+l4[i]
dict4        


In [ ]:
categories = list(dict4.keys())
values = list(dict4.values())

# Creating the bar plot
plt.bar(categories, values)

# Adding labels and title
plt.xlabel('Question Category')
plt.ylabel('Number of views')
plt.title('Number of views of each type of questions')

# Display the plot
plt.show()

In [ ]:
l7=new_questions['AnswerCount'].tolist()
l8=new_questions['PredictedClass'].tolist()
new_set=set(l8)
l9=list(new_set)
dict5=dict()
for element in l9:
    dict5[element]=0
n=len(l8)
for i in range(n):
    dict5[l8[i]]=dict5[l8[i]]+l7[i]
dict5  

In [ ]:
import matplotlib.pyplot as plt

# Data for the bar graph
categories =list(dict5.keys())
counts =list(dict5.values())

# Plotting the bar graph
plt.figure(figsize=(10, 6))
plt.bar(categories, counts, color=['blue', 'green', 'red','yellow','purple'])
plt.xlabel('Question Categories')
plt.ylabel('Number of Answers')
plt.title('Number of  total Answers of each type of questions')
plt.show()
